In [9]:
import requests
import numpy as np
import cv2
from matplotlib import pyplot as plt
import struct
import os

def get_image_from_url(url):
    response = requests.get(url)
    image_array = np.asarray(bytearray(response.content), dtype=np.uint8)
    color_image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    gray_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)
    return image_array, color_image, gray_image

def audio_to_binary(audio_file_path):
    with open(audio_file_path, 'rb') as audio_file:
        audio_data = audio_file.read()
    delimiter = b"<AUDIO_END>"
    size_info = struct.pack('<Q', len(audio_data))

    print(f"METADATA: {size_info + audio_data + delimiter}")
    return size_info + audio_data + delimiter

def binary_to_audio(binary_data, output_path):
    size = struct.unpack('<Q', binary_data[:8])[0]
    audio_data = binary_data[8:8+size]
    with open(output_path, 'wb') as audio_file:
        audio_file.write(audio_data)
    return True

def embed_audio_in_image(image, audio_path, output_path, bits_per_channel=2):
    if len(image.shape) == 3:
        height, width, channels = image.shape
    else:
        height, width = image.shape
        channels = 1
        image = np.expand_dims(image, axis=2)
    
    audio_binary = audio_to_binary(audio_path)
    bit_string = ''.join(format(byte, '08b') for byte in audio_binary)
    
    max_bits = height * width * channels * bits_per_channel
    if len(bit_string) > max_bits:
        raise ValueError(f"Imagen muy pequeña. Necesitas {len(bit_string)} bits, disponibles {max_bits}")
    
    mask = (0xFF << bits_per_channel) & 0xFF
    result_image = image.copy()
    
    bit_index = 0
    for i in range(height):
        for j in range(width):
            for k in range(channels):
                if bit_index < len(bit_string):
                    channel_bits = ""
                    for b in range(bits_per_channel):
                        if bit_index + b < len(bit_string):
                            channel_bits += bit_string[bit_index + b]
                        else:
                            channel_bits += "0"
                    
                    original_value = result_image[i, j, k] if channels > 1 else result_image[i, j, 0]
                    modified_value = (original_value & mask) | int(channel_bits, 2)
                    
                    if channels > 1:
                        result_image[i, j, k] = modified_value
                    else:
                        result_image[i, j, 0] = modified_value
                    
                    bit_index += bits_per_channel
                else:
                    break
            if bit_index >= len(bit_string):
                break
        if bit_index >= len(bit_string):
            break
    
    cv2.imwrite(output_path, result_image)
    return result_image

def extract_audio_from_image(image, output_audio_path, bits_per_channel=2):
    if len(image.shape) == 3:
        height, width, channels = image.shape
    else:
        height, width = image.shape
        channels = 1
        image = np.expand_dims(image, axis=2)
    
    extracted_bits = ""
    
    for i in range(height):
        for j in range(width):
            for k in range(channels):
                if channels > 1:
                    pixel_value = image[i, j, k]
                else:
                    pixel_value = image[i, j, 0]
                    
                lsb_bits = pixel_value & ((1 << bits_per_channel) - 1)
                extracted_bits += format(lsb_bits, f'0{bits_per_channel}b')
    
    binary_data = bytearray()
    for i in range(0, len(extracted_bits), 8):
        if i + 8 <= len(extracted_bits):
            byte = extracted_bits[i:i+8]
            binary_data.append(int(byte, 2))
    
    delimiter = b"<AUDIO_END>"
    delimiter_pos = binary_data.find(delimiter)
    if delimiter_pos == -1:
        raise ValueError("No se encontró audio válido en la imagen")
    
    audio_data = bytes(binary_data[:delimiter_pos])
    return binary_to_audio(audio_data, output_audio_path)

def get_image_capacity(image, bits_per_channel=2):
    if len(image.shape) == 3:
        height, width, channels = image.shape
    else:
        height, width = image.shape
        channels = 1
    
    max_bytes = (height * width * channels * bits_per_channel) // 8
    return max_bytes

def hide_audio_in_image_array(image_array, audio_file_path, bits_per_channel=2):
    audio_binary = audio_to_binary(audio_file_path)
    bit_string = ''.join(format(byte, '08b') for byte in audio_binary)
    
    flat_image = image_array.flatten()
    if len(bit_string) > len(flat_image) * bits_per_channel:
        raise ValueError("Imagen muy pequeña para el audio")
    
    mask = (0xFF << bits_per_channel) & 0xFF
    result_array = flat_image.copy()
    
    bit_index = 0
    for i in range(len(flat_image)):
        if bit_index < len(bit_string):
            channel_bits = ""
            for b in range(bits_per_channel):
                if bit_index + b < len(bit_string):
                    channel_bits += bit_string[bit_index + b]
                else:
                    channel_bits += "0"
            
            original_value = result_array[i]
            modified_value = (original_value & mask) | int(channel_bits, 2)
            result_array[i] = modified_value
            bit_index += bits_per_channel
        else:
            break
    
    return result_array.reshape(image_array.shape)

def extract_audio_from_image_array(image_array, output_path, bits_per_channel=2):
    flat_image = image_array.flatten()
    extracted_bits = ""
    
    for pixel_value in flat_image:
        lsb_bits = pixel_value & ((1 << bits_per_channel) - 1)
        extracted_bits += format(lsb_bits, f'0{bits_per_channel}b')
    
    binary_data = bytearray()
    for i in range(0, len(extracted_bits), 8):
        if i + 8 <= len(extracted_bits):
            byte = extracted_bits[i:i+8]
            binary_data.append(int(byte, 2))
    
    delimiter = b"<AUDIO_END>"
    delimiter_pos = binary_data.find(delimiter)
    if delimiter_pos == -1:
        raise ValueError("No se encontró audio válido")
    
    audio_data = bytes(binary_data[:delimiter_pos])
    return binary_to_audio(audio_data, output_path)


_, color_image, gray_image = get_image_from_url("https://images.pexels.com/photos/1996333/pexels-photo-1996333.jpeg?cs=srgb&dl=pexels-helenalopes-1996333.jpg&fm=jpg")

result_image = embed_audio_in_image(color_image, "sound.mp3", "output.png")

extract_audio_from_image(result_image, "extracted_audio.mp3")


capacity = get_image_capacity(color_image)
print(f"Capacidad: {capacity/1024:.2f} KB")

METADATA: b'c=\x03\x00\x00\x00\x00\x00\xff\xfb\xd0d\x00\x0f\xf0\x00\x00i\x00\x00\x00\x08\x00\x00\r \x00\x00\x01\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x04LAME3.100UUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUULAME3.100UUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUU\x18\x04\x06n\x03o86\x01s}\'5#\xe2\xd3\x85\x8cMD J\xf4\x8b\xbc\xdfH\r\xe8t\xe2\xc5\x87\xc7\r\x84\x00\xdc\x03N$\x18\xe5\tOl,\xf7\xc0\xcf\xb9X\xef`M\x92(\xcc]\xc4.\x06L\xeakh\x07\x14\x04o\xc4\x86\xe6\xb2fI\x06\x04N`\xe0\x86T@\x17C3\x93\x00\x8c\x90x\x01\xbb\x9a\x9aq\xe1\x99\xa3\x98B\x91\x85"\x99\x89Q\xa1\x02\x9a8y\x94\xc1\xa6\x16\x18\x984\xd4d\x82\xf9\x90\xd2FtY\x9be(sA\xa9\xda\xcfgMK\x9d\xd9\xf4o7\xa9\x95\x9c\xc6=\x8c\x99\x1eLn\'i\xd4\xc3GpX\x1b\